# NN BASELINE

In [1]:
#Copy metrics code from: https://www.kaggle.com/code/metric/event-detection-ap/notebook
!cp /kaggle/input/code-zzz/evaluation.py .

In [2]:
import glob
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import tensorflow as tf
from tqdm import tqdm
import pickle as pkl
import numpy as np
import logging
import polars as pl
logger=logging.getLogger()
import sys
from tqdm import tqdm
logger.setLevel(logging.INFO)
from tqdm import tqdm
print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))
if tf.test.is_gpu_available():
    print("TensorFlow está utilizando una GPU.")
    print("GPU utilizada:", tf.test.gpu_device_name())
else:
    print("TensorFlow no está utilizando una GPU.")

import matplotlib.pyplot as plt
import warnings

import gc
import pickle as pkl
from itertools import groupby
from datetime import timedelta

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


2.12.0
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]
TensorFlow está utilizando una GPU.
GPU utilizada: /device:GPU:0


In [3]:
from evaluation import score

## MODEL TARGET

It is used as the target whether the subject is sleeping or not (1 if they are and 0 if not). There are series with events that have 'step=null'. In these series, there are chunks of data where it is unknown where the previous or next event is. These data points are labeled as 'nan', and will be used only as features since the loss function uses a mask to compute the loss only on data points that do not have a 'nan' target.

Based on this target "sleep", events are constructed for evaluation and final submission using the 'get_event' function, which is being used in other notebooks.







In [4]:
TARGET="sleep"

In [5]:
PATH= "/kaggle/input/"

## PARAMS

In [6]:
LEARNING_RATE = 0.0001
STEPS_PER_EPOCH = 300
NUM_EPOCHS=15
WARMUP_STEPS = 64
GPU_BATCH_SIZE=4

In [7]:
OPTMIZER_BETA1=0.9
OPTMIZER_BETA2=0.98
OPTMIZER_EPSILON=1e-9

In [8]:
VALIDATE =False
## 5 folds in the fold_ids.pkl file (random folds of the 277 train series_ids, this way each full series is only in one fold). In this version only fold1 is used for training the model.

SAMPLE_NORMALIZE=True

In [9]:
class ModelSpec:
    def __init__(self,model_name, padding, use_temp,  model_type,drop_initial_date, sample_normalize,initial_hour=12, patch_size=12,model_dim=320,lstm_layers=5,use_tam_consider=False,TAM_RESHAPE=1,weight=1,STRIDE=0):
        self.model_name=model_name
        self.padding=padding
        self.use_temp=use_temp
        self.model_type=model_type
        self.drop_initial_date=drop_initial_date
        self.sample_normalize=sample_normalize
        self.initial_hour=initial_hour
        self.use_tam_consider=use_tam_consider
        self.TAM_RESHAPE=TAM_RESHAPE
        if self.model_type==LINEAR:
            self.CFG=CFG .copy()
        elif self.model_type==LINEAR_lstm_dropout:
            self.CFG=CFG_lstm_dropout.copy()
        elif self.model_type==CNN:
            self.CFG = CFG_CNN .copy()
        elif self.model_type==ATT:
            self.CFG=CFG_Att.copy()
            
        elif self.model_type==CONV1:
            self.CFG=CFGConv1.copy()
        elif self.model_type==CONV_5:
            self.CFG=CFGConv1.copy()
        elif self.model_type==CONV_6:
            self.CFG=CFGConv1.copy()
        elif self.model_type==GRU:
            self.CFG = CFG .copy()
        elif self.model_type==CONV1_LSTMDROPOUT:
              self.CFG=CFGConv1.copy()
        self.CFG[  "patch_size"] = patch_size
        
        self.CFG[  'model_dim']=model_dim
        self.CFG[   'model_num_lstm_layers'] =  lstm_layers
        if self.model_type!=ATT:
            self.CFG['feature_mixing_dim']=model_dim
        self.patch_size= patch_size
        if self.use_temp:
            self.dim = self.CFG["patch_size"]*2 +2 # 2 numeric features * patch size + 2 more features (sine and cosine of day time)
        else:
            self.dim =self.CFG["patch_size"]*2
            
        self.weight=weight
      
        self.STRIDE=STRIDE
        if   self.STRIDE!=0:
            self.CFG["stride"]=self.STRIDE
        

In [10]:
REPLACE="replace"
EXTEND="extend"
LINEAR="linear"
ATT="att"
CNN="cnn"
CONV1="conv1"
CONV_5="CONV_5"
CONV_6="CONV_6"
CONV1_LSTMDROPOUT="CONV1_LSTMDROPOUT"
CONV2="conv2"
GRU="GRU"
LINEAR_lstm_dropout="LINEAR_lstm_dropout"
DEFAULT_PATCH_SIZE = 12

In [11]:
CFG= {
   "num_epochs":NUM_EPOCHS,
   "steps_per_epoch":STEPS_PER_EPOCH,
   "patch_size":DEFAULT_PATCH_SIZE,
   "block_size":17280,
   "stride": 17280,
   'model_dim': 320,
   'time_mixing_dim':1440,
   'feature_mixing_dim':320,
   'model_num_heads': 6,
   'model_num_encoder_layers': 5,
   'model_num_lstm_layers': 5,
   'model_first_dropout': 0.5,
   'model_second_dropout':0.5,
   'model_encoder_dropout': 0.1,
   'model_mha_dropout': 0.0,
}

CFG_lstm_dropout = {
   "num_epochs":NUM_EPOCHS,
   "steps_per_epoch":STEPS_PER_EPOCH,
   "patch_size":10,
   "block_size":17280,
   "stride": 17280,
   'model_dim': 160,
   'time_mixing_dim':1440,
   'feature_mixing_dim':160,
   'model_num_heads': 6,
   'model_num_encoder_layers': 5,
   'model_num_lstm_layers': 5,
   'model_first_dropout': 0.5,
   'model_second_dropout':0.5,
   'model_encoder_dropout': 0.1,
   'model_mha_dropout': 0.0,
}
CFG_Att  = {
   "num_epochs":NUM_EPOCHS,
   "steps_per_epoch":STEPS_PER_EPOCH,
   "patch_size":10,
   "block_size":17280,
   "stride": 17280,
   'model_dim': 160,
   'time_mixing_dim':1440,
   'feature_mixing_dim':80,
   'model_num_heads': 6,
   'model_num_encoder_layers': 5,
   'model_num_lstm_layers': 2,
   'model_first_dropout': 0.5,
   'model_second_dropout':0.5,
   'model_encoder_dropout': 0.1,
   'model_mha_dropout': 0.0,
   'model_transformer_layers':3,
   "transformer_dim":320
}

CFG_CNN = {
   "num_epochs":NUM_EPOCHS,
   "steps_per_epoch":STEPS_PER_EPOCH,
   "patch_size":10,
   "block_size":17280,
   "stride": 17280,
   'model_dim': 160,
   'time_mixing_dim':1440,
   'feature_mixing_dim':160,
   'model_num_heads': 6,
   'model_num_encoder_layers': 5,
   'model_num_lstm_layers': 2,
   'model_cnn_layers':3,
   'model_first_dropout': 0.5,
   'model_second_dropout':0.5,
   'model_encoder_dropout': 0.1,
   'model_mha_dropout': 0.0,
}

CFGConv1 = {
   "num_epochs":NUM_EPOCHS,
   "steps_per_epoch":STEPS_PER_EPOCH,
   "patch_size":1,
   "block_size":17280,
   "stride": 17280,
   'model_dim': 160,
   'time_mixing_dim':1440,
   'feature_mixing_dim':160,
   'model_num_heads': 6,
   'model_num_encoder_layers': 3,
   'model_num_lstm_layers': 5,
   'model_first_dropout': 0.2,
   'model_second_dropout':0.5,
   'model_encoder_dropout': 0.1,
   'model_mha_dropout': 0.0,
}
CFG_CONV1_LSTMDROPOUT  = {
   "num_epochs":NUM_EPOCHS,
   "steps_per_epoch":STEPS_PER_EPOCH,
   "patch_size":1,
   "block_size":17280,
   "stride": 17280,
   'model_dim': 160,
   'time_mixing_dim':1440,
   'feature_mixing_dim':160,
   'model_num_heads': 6,
   'model_num_encoder_layers': 3,
   'model_num_lstm_layers': 5,
   'model_first_dropout': 0.2,
   'model_second_dropout':0.5,
   'model_encoder_dropout': 0.1,
   'model_mha_dropout': 0.0,
}

In [12]:

if VALIDATE:
    file = "/kaggle/input/zzz-data-train/data_train.parquet"
else:
    file="/kaggle/input/child-mind-institute-detect-sleep-states/test_series.parquet"

In [13]:
dict_ids=pkl.load(open("/kaggle/input/zzz-data-train/dict_ids.pkl","rb"))
swapped_dict={value:key for key,value in dict_ids.items()}
fold_ids=pkl.load(open("/kaggle/input/zzz-data-train/fold_ids.pkl","rb"))

In [14]:
ids_test  = pd.read_parquet(file, columns=['series_id'])
ids_test = ids_test.series_id.unique()
ids_test 

array(['038441c925bb', '03d92c9f6f8a', '0402a003dae9'], dtype=object)

In [15]:
import os
import pyarrow.parquet as pq
import pandas as pd
from tqdm import tqdm 
if not VALIDATE:
   
    # Abre el archivo parquet
    pf = pq.ParquetFile(file)
    !mkdir series
    # Cantidad de grupos de filas en el archivo parquet
    num_row_groups = pf.num_row_groups

    # Procesar grupo por grupo
    for r in tqdm(range(num_row_groups)):
        # Leer un grupo de filas
        table = pf.read_row_group(r, columns=["anglez", "enmo", "series_id","step","sleep","timestamp",])
        df = table.to_pandas()

        # Procesa y guarda cada series_id en su archivo
        for id_, subset in df.groupby("series_id"):
            # Determina la ruta del archivo
            file_path = f"series/series_{id_}.parquet"

            if os.path.exists(file_path):
                # Si el archivo ya existe, lee el contenido existente y concatena
                existing_df = pd.read_parquet(file_path)
                combined_df = pd.concat([existing_df, subset], axis=0)
                combined_df.to_parquet(file_path, index=False)
            else:
                # Si no, solo guarda el subconjunto
                subset.to_parquet(file_path, index=False)

100%|██████████| 1/1 [00:00<00:00, 52.24it/s]


In [16]:
numeric_features= ["anglez", "enmo"]

In [17]:

class EncoderConv1(tf.keras.Model):
    def __init__(self,j):
        super().__init__()

        self.first_linear = tf.keras.layers.Dense(CFGConv1['time_mixing_dim'])
        self.second_linear = tf.keras.layers.Dense(CFGConv1['feature_mixing_dim'])


        self.conv1=tf.keras.layers.Conv1D(32*(2**j),3,strides=1,padding="same",)
        self.conv2=tf.keras.layers.Conv1D(32*(2**j),3,strides=1,padding="same",)
        self.conv3=tf.keras.layers.Conv1D(32*(2**j),3,strides=1,padding="same",)
        #self.norm=tf.keras.layers.BatchNormalization()
        self.max_pool=tf.keras.layers.AveragePooling1D( pool_size=2)
        self.dropout = tf.keras.layers.Dropout(0.2)


    def call(self, x, training=None):



        res=x

        x=self.conv1(x)
        x=self.conv2(x)
        x=self.conv3(x)
        #x=self.norm(x)
        x=tf.keras.layers.ReLU(  )(x)
        #x = self.dropout( x)
        x= self.max_pool(x)

        return x


class ModelConv1(tf.keras.Model):
    def __init__(self):
        super().__init__()

        self.encoders = [EncoderConv1(i+1) for i in range( CFGConv1['model_num_encoder_layers'])]
        self.lstm_layers = [tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(CFGConv1['model_dim'], return_sequences=True)) for _ in range(CFGConv1['model_num_lstm_layers'])]

        self.linear = tf.keras.layers.Dense(64)

        #self.first_dropout = tf.keras.layers.Dropout(0.2)
        self.last_linear = tf.keras.layers.Dense(2)

    def call(self, x):




        for i in range(CFGConv1['model_num_encoder_layers']): x = self.encoders[i](x)

        for i in range(CFGConv1['model_num_lstm_layers']):x= self.lstm_layers[i](x)
        x =self.linear(x)
        x = self.last_linear(x)
        x = tf.nn.sigmoid(x)
        return x


In [18]:
class EncoderConv6(tf.keras.Model):
    def __init__(self,j):
        super().__init__()
        self.j=j
        self.first_linear = tf.keras.layers.Dense(CFGConv1['time_mixing_dim'])
        self.second_linear = tf.keras.layers.Dense(CFGConv1['feature_mixing_dim'])


        self.conv1=tf.keras.layers.Conv1D(32*(2**j),3,strides=1,padding="same",)
        self.conv2=tf.keras.layers.Conv1D(32*(2**j),3,strides=1,padding="same",)
        self.conv3=tf.keras.layers.Conv1D(32*(2**j),3,strides=1,padding="same",)
        #self.norm=tf.keras.layers.BatchNormalization()
        if self.j==1:
          POOL=2
        else:
          POOL=3
        self.max_pool=tf.keras.layers.AveragePooling1D( pool_size= POOL)
        self.dropout = tf.keras.layers.Dropout(0.2)


    def call(self, x, training=None):



        res=x

        x=self.conv1(x)
        x=self.conv2(x)
        x=self.conv3(x)
        #x=self.norm(x)
        x=tf.keras.layers.ReLU(  )(x)
        #x = self.dropout( x)
        if self.j<3:
          x= self.max_pool(x)

        return x

class ModelConv6(tf.keras.Model):
    def __init__(self):
        super().__init__()

        self.encoders = [EncoderConv6(i+1) for i in range( CFGConv1['model_num_encoder_layers'])]
        self.lstm_layers = [tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(CFGConv1['model_dim'], return_sequences=True)) for _ in range(CFGConv1['model_num_lstm_layers'])]

        self.linear = tf.keras.layers.Dense(64)

        #self.first_dropout = tf.keras.layers.Dropout(0.2)
        self.last_linear = tf.keras.layers.Dense(2)

    def call(self, x):




        for i in range(CFGConv1['model_num_encoder_layers']): x = self.encoders[i](x)

        for i in range(CFGConv1['model_num_lstm_layers']):x= self.lstm_layers[i](x)
        x =self.linear(x)
        x = self.last_linear(x)
        x = tf.nn.sigmoid(x)
        return x

    
    
class EncoderConv5(tf.keras.Model):
    def __init__(self,j):
        super().__init__()
        self.j=j
        self.first_linear = tf.keras.layers.Dense(CFGConv1['time_mixing_dim'])
        self.second_linear = tf.keras.layers.Dense(CFGConv1['feature_mixing_dim'])


        self.conv1=tf.keras.layers.Conv1D(32*(2**j),3,strides=1,padding="same",)
        self.conv2=tf.keras.layers.Conv1D(32*(2**j),3,strides=1,padding="same",)
        self.conv3=tf.keras.layers.Conv1D(32*(2**j),3,strides=1,padding="same",)
        #self.norm=tf.keras.layers.BatchNormalization()
        self.max_pool=tf.keras.layers.AveragePooling1D( pool_size=2)
        self.dropout = tf.keras.layers.Dropout(0.2)


    def call(self, x, training=None):



        res=x

        x=self.conv1(x)
        x=self.conv2(x)
        x=self.conv3(x)
        #x=self.norm(x)
        x=tf.keras.layers.ReLU(  )(x)
        #x = self.dropout( x)
        if self.j<3:
          x= self.max_pool(x)

        return x

class ModelConv5(tf.keras.Model):
    def __init__(self):
        super().__init__()

        self.encoders = [EncoderConv5(i+1) for i in range( CFGConv1['model_num_encoder_layers'])]
        self.lstm_layers = [tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(CFGConv1['model_dim'], return_sequences=True)) for _ in range(CFGConv1['model_num_lstm_layers'])]

        self.linear = tf.keras.layers.Dense(64)

        #self.first_dropout = tf.keras.layers.Dropout(0.2)
        self.last_linear = tf.keras.layers.Dense(2)

    def call(self, x):




        for i in range(CFGConv1['model_num_encoder_layers']): x = self.encoders[i](x)

        for i in range(CFGConv1['model_num_lstm_layers']):x= self.lstm_layers[i](x)
        x =self.linear(x)
        x = self.last_linear(x)
        x = tf.nn.sigmoid(x)
        return x

In [19]:

class  EncoderConv1_LSTMDROPOUT(tf.keras.Model):
    def __init__(self,j):
        super().__init__()

        self.first_linear = tf.keras.layers.Dense(CFG_CONV1_LSTMDROPOUT['time_mixing_dim'])
        self.second_linear = tf.keras.layers.Dense(CFG_CONV1_LSTMDROPOUT['feature_mixing_dim'])


        self.conv1=tf.keras.layers.Conv1D(32*(2**j),3,strides=1,padding="same",)
        self.conv2=tf.keras.layers.Conv1D(32*(2**j),3,strides=1,padding="same",)
        self.conv3=tf.keras.layers.Conv1D(32*(2**j),3,strides=1,padding="same",)
        #self.norm=tf.keras.layers.BatchNormalization()
        self.max_pool=tf.keras.layers.AveragePooling1D( pool_size=2)
        self.dropout = tf.keras.layers.Dropout(0.2)


    def call(self, x, training=None):



        res=x

        x=self.conv1(x)
        x=self.conv2(x)
        x=self.conv3(x)
        #x=self.norm(x)
        x=tf.keras.layers.ReLU(  )(x)
        #x = self.dropout( x)
        x= self.max_pool(x)

        return x

class ModelConv1_LSTMDROPOUT(tf.keras.Model):
    def __init__(self):
        super().__init__()

        self.encoders = [EncoderConv1_LSTMDROPOUT(i+1) for i in range( CFG_CONV1_LSTMDROPOUT['model_num_encoder_layers'])]
        self.lstm_layers = [tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(CFG_CONV1_LSTMDROPOUT['model_dim'], return_sequences=True,dropout=0.2)) for _ in range(CFG_CONV1_LSTMDROPOUT['model_num_lstm_layers'])]

        self.linear = tf.keras.layers.Dense(64)

        #self.first_dropout = tf.keras.layers.Dropout(0.2)
        self.last_linear = tf.keras.layers.Dense(2)

    def call(self, x):




        for i in range(CFG_CONV1_LSTMDROPOUT['model_num_encoder_layers']): x = self.encoders[i](x)

        for i in range(CFG_CONV1_LSTMDROPOUT['model_num_lstm_layers']):x= self.lstm_layers[i](x)
        x =self.linear(x)
        x = self.last_linear(x)
        x = tf.nn.sigmoid(x)
        return x

In [20]:

class Encoder(tf.keras.Model):
    def __init__(self,CFG):
        super().__init__()

        self.first_linear = tf.keras.layers.Dense(CFG['time_mixing_dim'])
        self.second_linear = tf.keras.layers.Dense(CFG['feature_mixing_dim'])

        self.add = tf.keras.layers.Add()

        self.first_dropout = tf.keras.layers.Dropout(CFG['model_first_dropout'])

        self.second_dropout = tf.keras.layers.Dropout(CFG['model_second_dropout'])
      

        


    def call(self, x, training=None):
        
        
        # time mixing
        """time_mixing= tf.transpose(x,perm=[0,2,1])
        time_mixing  = self.first_linear(time_mixing )

        time_mixing =tf.keras.layers.ReLU( )(time_mixing)

        time_mixing  = self.first_dropout(time_mixing )

        time_mixing =tf.transpose(  time_mixing,perm=[0,2,1])

        x=self.add ([x,time_mixing])"""

        features_mixing = self.second_linear(x )

        features_mixing  = tf.keras.layers.ReLU(  )(features_mixing)

        features_mixing   = self.second_dropout( features_mixing )

        x=self.add ([x,features_mixing ])
        return x

class Model(tf.keras.Model):
    def __init__(self,CFG):
        super().__init__()
        self.CFG=CFG
        self.encoders = [Encoder( self.CFG) for i in range( self.CFG['model_num_encoder_layers'])]

        self.lstm_layers = [tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(self.CFG['model_dim'], return_sequences=True)) for _ in range(self.CFG['model_num_lstm_layers'])]

        self.first_linear = tf.keras.layers.Dense(self.CFG['feature_mixing_dim'])
        self.first_dropout = tf.keras.layers.Dropout(0.1)
        self.last_linear = tf.keras.layers.Dense(2)

    def call(self, x):


        x=self.first_linear(x)
        x=self.first_dropout(x)
        for i in range(self.CFG['model_num_encoder_layers']): x = self.encoders[i](x)

        for i in range(self.CFG['model_num_lstm_layers']):x= self.lstm_layers[i](x)
        x = self.last_linear(x)
        x = tf.nn.sigmoid(x)
        return x

In [21]:
class EncoderGRU(tf.keras.Model):
    def __init__(self,CFG):
        super().__init__()

        self.first_linear = tf.keras.layers.Dense(CFG['time_mixing_dim'])
        self.second_linear = tf.keras.layers.Dense(CFG['feature_mixing_dim'])

        self.add = tf.keras.layers.Add()

        self.first_dropout = tf.keras.layers.Dropout(CFG['model_first_dropout'])

        self.second_dropout = tf.keras.layers.Dropout(CFG['model_second_dropout'])





    def call(self, x, training=None):




        features_mixing = self.second_linear(x )

        features_mixing  = tf.keras.layers.ReLU(  )(features_mixing)

        features_mixing   = self.second_dropout( features_mixing )

        x=self.add ([x,features_mixing ])
        return x

class ModelGRU(tf.keras.Model):
    def __init__(self,CFG):
        super().__init__()
        self.CFG=CFG
        self.encoders = [EncoderGRU(CFG) for i in range( CFG['model_num_encoder_layers'])]

        self.lstm_layers = [tf.keras.layers.Bidirectional(tf.keras.layers.GRU(CFG['model_dim'], return_sequences=True)) for _ in range(CFG['model_num_lstm_layers'])]

        self.first_linear = tf.keras.layers.Dense(CFG['feature_mixing_dim'])
        self.first_dropout = tf.keras.layers.Dropout(0.1)
        self.last_linear = tf.keras.layers.Dense(2)

    def call(self, x):


        x=self.first_linear(x)
        x=self.first_dropout(x)
        for i in range(self.CFG['model_num_encoder_layers']): x = self.encoders[i](x)

        for i in range(self.CFG['model_num_lstm_layers']):x= self.lstm_layers[i](x)
        x = self.last_linear(x)
        x = tf.nn.sigmoid(x)
        return x

In [22]:


class Encoder_lstm_dropout(tf.keras.Model):
    def __init__(self):
        super().__init__()

        self.first_linear = tf.keras.layers.Dense(CFG_lstm_dropout['time_mixing_dim'])
        self.second_linear = tf.keras.layers.Dense(CFG_lstm_dropout['feature_mixing_dim'])

        self.add = tf.keras.layers.Add()

        self.first_dropout = tf.keras.layers.Dropout(CFG_lstm_dropout['model_first_dropout'])

        self.second_dropout = tf.keras.layers.Dropout(CFG_lstm_dropout['model_second_dropout'])





    def call(self, x, training=None):




        features_mixing = self.second_linear(x )

        features_mixing  = tf.keras.layers.ReLU(  )(features_mixing)

        features_mixing   = self.second_dropout( features_mixing )

        x=self.add ([x,features_mixing ])
        return x

class Model_lstm_dropout(tf.keras.Model):
    def __init__(self):
        super().__init__()

        self.encoders = [Encoder_lstm_dropout() for i in range( CFG_lstm_dropout['model_num_encoder_layers'])]

        self.lstm_layers = [tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(CFG_lstm_dropout['model_dim'], return_sequences=True,dropout=0.2)) for _ in range(CFG_lstm_dropout['model_num_lstm_layers'])]

        self.first_linear = tf.keras.layers.Dense(CFG_lstm_dropout['feature_mixing_dim'])
        self.first_dropout = tf.keras.layers.Dropout(0.1)
        self.last_linear = tf.keras.layers.Dense(2)

    def call(self, x):


        x=self.first_linear(x)
        x=self.first_dropout(x)
        for i in range(CFG_lstm_dropout['model_num_encoder_layers']): x = self.encoders[i](x)

        for i in range(CFG_lstm_dropout['model_num_lstm_layers']):x= self.lstm_layers[i](x)
        x = self.last_linear(x)
        x = tf.nn.sigmoid(x)
        return x

##  GET AND BUILD DATA

The training data is constructed in another notebook exactly the same way as is test data here:

In [23]:
events=pd.read_csv("/kaggle/input/child-mind-institute-detect-sleep-states/train_events.csv").dropna(subset="timestamp")
events_check=pd.read_csv(f"{PATH}/child-mind-institute-detect-sleep-states/train_events.csv").dropna(subset="timestamp")

In [24]:
def sample_normalize(sample):
    mean = tf.math.reduce_mean(sample,axis=0)
    std = tf.math.reduce_std(sample,axis=0)
    sample = tf.math.divide_no_nan(sample-mean, std)

    return sample.numpy()

In [25]:
def truncate_days(df_,id_):
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        swaped_dict={value:key for key , value in dict_ids.items()}
        events_=events_check.loc[events_check.series_id==swaped_dict[id_]]
        events_["timestamp"] =pd.to_datetime(events_["timestamp"].str[:19] )
        prev_lenght=len(df_)
        df_=df_.loc[df_.timestamp.dt.date<= events_.timestamp.dt.date.max()]
        return df_

In [26]:

def drop_initial_date(df__,model):

            if VALIDATE:
                df__=df__.loc[~(df__.timestamp.diff()==timedelta(seconds=0))]
            inital_time=df__.iloc[0].timestamp
            
            if model.padding==REPLACE:
                intial_padding = (inital_time.hour*60+inital_time.minute - 60*model.initial_hour)*12
            elif model.padding==EXTEND:
                intial_padding = (inital_time.hour*60+inital_time.minute)*model.initial_hour +12*60*12
            if intial_padding >0:
                df_ini=pd.DataFrame(np.zeros((intial_padding,len(df__.columns))),columns=df__.columns)
                df_ini["sleep"]=np.nan
                return pd.concat([df_ini,df__])
            else:


                return df__.iloc[-intial_padding:]

In [27]:
from tensorflow.python.distribute.distribute_lib import def_function

pd.options.mode.chained_assignment = None 

def process_df(df_):
            df_["timestamp"]=pd.to_datetime(df_["timestamp"].str[:19])    
            df_["minute"]=   df_["timestamp"].dt.hour*60+ df_["timestamp"].dt.minute
            df_["sine"]=np.sin(df_["minute"]*np.pi*2/ 1440)
            df_["cosine"]=np.cos(df_["minute"]*np.pi*2/ 1440)
            return df_
        
        
        

def read_data(id_, model ):
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore", category=RuntimeWarning)
        features={}
        targets={}
        descrp={}
        steps_={}
        preds_dic_len={}
        
        features[id_]=[]
        targets[id_]=[]
        steps_[id_]=[]
        df_=pd.read_parquet(f"/kaggle/working/series/series_{id_}.parquet")
        #df_["timestamp"] = pd.to_datetime(df_["timestamp"] )
        df_=process_df(df_)
      
        
        if len(df_)>0:
            if model. drop_initial_date:
              df_=drop_initial_date(df_,model)
            df_=df_.reset_index(drop=True)
            preds_dic_len[id_]=len(df_)
            steps = range(0,len(df_),model.CFG["stride"])
            steps= [step for step in steps if step<len(df_)]
            descrp[id_]={"steps":len(steps),"length_df":len(df_)}
            

            for step in steps:
                sample_=df_.loc[step:step+model.CFG["block_size"]-1,:]
                feats=sample_.loc[:,numeric_features].values
                if model.sample_normalize:
                    feats=sample_normalize(feats)
                sine_=sample_["sine"].values.reshape(-1,1)
                cosine_=sample_["cosine"].values.reshape(-1,1)
             

                if len(feats)<model.CFG["block_size"]:
                      padding = model.CFG["block_size"] - len(feats)
                      padding_values = np.zeros((padding, feats.shape[1]))
                      padding_sine = np.zeros((padding, sine_.shape[1]))
                      padding_cosine = np.zeros((padding, cosine_.shape[1]))
                      
                        
                      feats = np.vstack([feats, padding_values])
                      sine_ = np.vstack([sine_, padding_sine])
                      cosine_ = np.vstack([cosine_, padding_cosine])
                      

                feats=feats.reshape(-1,model.CFG["patch_size"]*2).astype(np.float32)
                sine_=sine_.reshape(-1,model.CFG["patch_size"],1).mean(axis=1).reshape(-1,1)
                cosine_=cosine_.reshape(-1,model.CFG["patch_size"],1).mean(axis=1).reshape(-1,1)
  
                
                if model.use_temp:
                    features[id_].append(np.concatenate((feats,sine_,cosine_),axis=1))

                else:
                    features[id_].append(feats)



                steps_[id_].append(step)

        
                    
                del sample_
                    
            gc.collect()
            

    return    features,   descrp,  steps_, df_


def read_data_validate(id_, model ):
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore", category=RuntimeWarning)
        features={}
        targets={}
        descrp={}
        steps_={}
        preds_dic_len={}
        lenghts={}
        steps_save={} 
        features[id_]=[]
        targets[id_]=[]
        steps_[id_]=[]
        df_=pd.read_parquet(f"/kaggle/input/zzz-series/series/series_{id_}.parquet")
        df_["timestamp"] = pd.to_datetime(df_["timestamp"] )
        #df_=process_df(df_)
        df_= truncate_days(df_,id_)
        #print(len(df_))
        events_=events.loc[events.series_id==swapped_dict[id_]].dropna(subset="timestamp")
        if len(df_)>0 and len(events_)>0:
            if model. drop_initial_date:
              df_=drop_initial_date(df_,model)
            df_=df_.reset_index(drop=True)
            preds_dic_len[id_]=len(df_)
            steps = range(0,len(df_),model.CFG["stride"])
            steps= [step for step in steps if step<len(df_)]
            descrp[id_]={"steps":len(steps),"length_df":len(df_)}
            

            for step in steps:
                sample_=df_.loc[step:step+model.CFG["block_size"]-1,:]
                feats=sample_.loc[:,numeric_features].values
                if model.sample_normalize:
                    feats=sample_normalize(feats)
                sine_=sample_["sine"].values.reshape(-1,1)
                cosine_=sample_["cosine"].values.reshape(-1,1)
                target_=sample_.loc[:,TARGET].values.reshape(-1,1)

                if len(feats)<model.CFG["block_size"]:
                      padding = model.CFG["block_size"] - len(feats)
                      padding_values = np.zeros((padding, feats.shape[1]))
                      padding_sine = np.zeros((padding, sine_.shape[1]))
                      padding_cosine = np.zeros((padding, cosine_.shape[1]))
                      padding_target = np.empty((padding, cosine_.shape[1])) *np.nan
                        
                      feats = np.vstack([feats, padding_values])
                      sine_ = np.vstack([sine_, padding_sine])
                      cosine_ = np.vstack([cosine_, padding_cosine])
                      target_ = np.vstack([target_, padding_target])

                feats=feats.reshape(-1,model.CFG["patch_size"]*2).astype(np.float32)
                sine_=sine_.reshape(-1,model.CFG["patch_size"],1).mean(axis=1).reshape(-1,1)
                cosine_=cosine_.reshape(-1,model.CFG["patch_size"],1).mean(axis=1).reshape(-1,1)
                targets_=np.nanmean(target_.reshape(-1,model.CFG["patch_size"],1),axis=1).round().astype(np.float16)
                #if np.isnan(targets_).sum()!=len(targets_):
                if True:
                    if model.use_temp:
                        features[id_].append(np.concatenate((feats,sine_,cosine_),axis=1))

                    else:
                        features[id_].append(feats)
                    


                    steps_[id_].append(step)

               
                else:
                    df_=df_.loc[(df_.index<step)|(df_.index>=step+model.CFG["block_size"])]
                    
                del sample_
                    
            gc.collect()
            

    return    features,   descrp,  steps_, df_

In [28]:

ce = tf.keras.losses.BinaryCrossentropy(reduction='none')

def loss_function(real, output, name='loss_function'):

    # a mask is applied to not compute nan target
    mask = tf.math.logical_not(tf.math.is_nan(real))



    y_true = tf.boolean_mask(real, mask)
    y_pred = tf.boolean_mask(output, mask)
    tf.debugging.check_numerics(y_true, message="NaNs in 'real'")
    tf.debugging.check_numerics(y_pred, message="NaNs in 'output'")
    loss = ce(tf.expand_dims(y_true,axis=-1),tf.expand_dims(y_pred,axis=-1))
    tf.debugging.check_numerics(loss, message="NaNs in 'loss'")

    return tf.reduce_mean(loss)
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, initial_lr, warmup_steps=1):
        super(CustomSchedule, self).__init__()

        self.initial_lr = tf.cast(initial_lr, tf.float32)
        self.warmup_steps = tf.cast(warmup_steps, tf.float32)

    def __call__(self, step):
        step = tf.cast(step, tf.float32)
        return tf.math.minimum(self.initial_lr, self.initial_lr * (step/self.warmup_steps))


In [29]:
import gc
from scipy.signal import find_peaks
def get_event(df,col="pred"):
        lstCV = zip(df.series_id, df[col])
        lstPOI = []
        for (c, v), g in groupby(lstCV, lambda cv:
                                (cv[0], cv[1]!=0 and not pd.isnull(cv[1]))):
            llg = sum(1 for item in g)
            if v is False:
                lstPOI.extend([0]*llg)
            else:
                lstPOI.extend(['onset']+(llg-2)*[0]+['wakeup'] if llg > 1 else [0])
        return lstPOI
    
            
def get_events(pred,data_,idx,min_interval=30,patch_size= 12,h=1.3429635e-07,d=30,TAM_CONSIDER=None) :
   

       
        
    test_ds=data_

    
    


    days = len(pred)/(17280/100)

    submission=pd.DataFrame(columns=["step",	"event",	"series_id",	"score"])
    #candidates_onset = np.argsort(-pred[:,0])
    #candidates_wakeup =  np.argsort(-pred[:,1])
    #n_add=max(1,round(days))

    #added_onset=[]
    #added_wakeup=[]
    #disponibles=list(candidates_onset.copy())
    #while len(added_onset)<n_add and len(disponibles)>0:
    #    actual = disponibles.pop(0)
    #    added_onset.append(actual)
    #    disponibles = [x for x in disponibles if abs(x - actual) >= min_interval]

    #disponibles=list(candidates_wakeup.copy())
    #while len( added_wakeup)<n_add and len(disponibles)>0:
    #    actual = disponibles.pop(0)
    #    added_wakeup.append(actual)
    #    disponibles = [x for x in disponibles if abs(x - actual) >= min_interval]
    #added_onset=np.array(  added_onset)
    #added_wakeup=np.array(    added_wakeup)
    added_onset= np.array(find_peaks( pred[:,0],height=h,distance=d)[0])
    added_wakeup= np.array(find_peaks( pred[:,1],height=h,distance=d)[0])
    onset = test_ds[['step']].iloc[np.clip(added_onset* TAM_CONSIDER,0,len( test_ds)-1)].astype(np.int32)
    onset['event'] = 'onset'
    onset['series_id'] =  idx
    onset['score']= pred[added_onset,0]
    wakeup = test_ds[['step']].iloc[np.clip(added_wakeup* TAM_CONSIDER,0,len( test_ds)-1)].astype(np.int32)
    wakeup['event'] = 'wakeup'
    wakeup['series_id'] = idx
    wakeup['score']= pred[added_wakeup,1]
    submission = pd.concat([submission,onset,wakeup],axis=0)

    return submission
def get_preds_df( val_ids,preds_dict,data,smoothing_length=480,TAM_CONSIDER=None ):
    submision=[]
    for id_ in tqdm(val_ids):
                if len(preds_dict[id_])>0:

                    preds=preds_dict[id_]

                    df_=data.loc[data.series_id==id_]
                    df_=truncate_days(df_,id_)
                    data_=df_
                    if DROP_INITIAL_DATE  and not  len(data)==450:
                      data_=drop_initial_date(data_)




                    submision.append(get_events(data_,id_,preds,TAM_CONSIDER=TAM_CONSIDER))
    submision= pd.concat(submision)
    submision["step"]=submision["step"].astype(np.float32)
    return submision
def get_df(final_preds,df_,model,TAM_CONSIDER):
    
    #final_preds=np.tile(final_preds.reshape(-1,1),model.CFG["patch_size"]).reshape(-1,1)
    if model.STRIDE!=0:
        offset= model.CFG["block_size"]- model.STRIDE
        #print(f"Using STRIDE = {model.STRIDE}, offset = {   offset}")
        final_preds_stride= []
       
        for preds__ in final_preds:
        
            if len(final_preds_stride)==0:
                final_preds_stride.append(preds__ )
            elif len(final_preds_stride)==1:
                arr= np.concatenate( [ final_preds_stride[len( final_preds_stride)-1][model.STRIDE:,0].reshape(-1,1), preds__ [:len(preds__ )-model.STRIDE,0].reshape(-1,1)  ],axis=1)
                final_preds_stride[len( final_preds_stride)-1][model.STRIDE:,0] = np.nanmean(arr,axis=1)
                arr=np.concatenate ([final_preds_stride[len(final_preds_stride)-1][model.STRIDE:,1].reshape(-1,1), preds__ [:len(preds__ )-model.STRIDE,1].reshape(-1,1)  ],axis=1)
                final_preds_stride[len( final_preds_stride)-1][model.STRIDE:,1] = np.nanmean( arr,axis=1)
                final_preds_stride.append(preds__[len(preds__)-model.STRIDE:])
                del arr
            else:
                
                arr= np.concatenate( [ final_preds_stride[len( final_preds_stride)-1][model.STRIDE- offset:,0].reshape(-1,1), preds__ [:len(preds__ )-model.STRIDE,0].reshape(-1,1)  ],axis=1)
                final_preds_stride[len( final_preds_stride)-1][model.STRIDE- offset:,0] = np.nanmean(arr,axis=1)
                arr=np.concatenate ([final_preds_stride[len(final_preds_stride)-1][model.STRIDE- offset:,1].reshape(-1,1), preds__ [:len(preds__ )-model.STRIDE,1].reshape(-1,1)  ],axis=1)
                final_preds_stride[len( final_preds_stride)-1][model.STRIDE- offset:,1] = np.nanmean( arr,axis=1)
                final_preds_stride.append(preds__[len(preds__)-model.STRIDE:])
                del arr
            del preds__
            
          

      
        final_preds = final_preds_stride   
        del final_preds_stride
        gc.collect()  
    final_preds=np.concatenate(final_preds,axis=0)
    final_preds=final_preds[:TAM_CONSIDER * len(final_preds)//TAM_CONSIDER]

    if model.use_tam_consider:
        
        final_preds_r=np.mean(  final_preds.reshape(-1,TAM_CONSIDER,2),axis=1)
        del final_preds
        gc.collect()
        return  final_preds_r*model.weight
    else:
        return  final_preds*model.weight
      
        


In [30]:
import matplotlib.pyplot as plt

In [31]:

def predict_model(models,TAM_CONSIDER=5):
    name_save="submision.csv"
    for model_ in models:
        if model_.model_type==LINEAR:
            model=Model(model_.CFG)

        elif model_.model_type==ATT:
            model=ModelATT(model_.CFG)
        elif model_.model_type==CNN:
            model=ModelCNN(model_.CFG)
        elif model_.model_type==CONV1:
            model=ModelConv1()
        elif  model_.model_type==LINEAR_lstm_dropout:
            model=Model_lstm_dropout()
        elif model_.model_type==CONV1_LSTMDROPOUT:
            model=ModelConv1_LSTMDROPOUT()
        elif model_.model_type==CONV_6:
            model=ModelConv6()
        elif model_.model_type==CONV_5:
            model=ModelConv5()
        elif model_.model_type== GRU:
            model=ModelGRU(model_.CFG)
       
        model.load_weights(model_.model_name)
        model_.model=model
        
    submisions=[]
    submision=None
    df_total=None
    dfs_save=[]
    #print(ids_test)
    for id_ in tqdm(ids_test):
 
        model_= models[0]
        if VALIDATE:
            features_test,   descrp_test,  steps_test, df_ = read_data_validate(id_,model= model_)
        else:
            features_test,   descrp_test,  steps_test, df_ = read_data(id_,model=  model_)
        features_use={}
        features_use[( model_.initial_hour, model_.patch_size)] = [features_test,   descrp_test,  steps_test, df_]
        if len(df_)>0:
            
            
            matrix=np.stack(features_test[id_]).reshape((-1, model_. CFG['block_size'] //  model_.CFG['patch_size'],  model_. dim))
        
            preds_total =  model_.model.predict(matrix,verbose=0)
           
            if model_.TAM_RESHAPE!=1:
                        preds_total =np.tile(   preds_total ,model_.TAM_RESHAPE).reshape(    preds_total .shape[0],    preds_total .shape[1]*model_.TAM_RESHAPE,2)
                 
     
            df_total=  get_df(preds_total,df_,model_,TAM_CONSIDER=TAM_CONSIDER)
       
            if False:
                fig=plt.figure()
                plt.plot(np.arange(len(df_total.reshape(-1,2))),df_total.reshape(-1,2)[:,0])
                plt.title(f"ID: {id_}, {model_.model_name}")
                plt.show()

            for model_ in models[1:]:
                    if (model_.initial_hour, model_.patch_size) in features_use.keys():
                        features_test,   descrp_test,  steps_test, df_ =  features_use[(model_.initial_hour, model_.patch_size)] 
                    else:
                        
                        if VALIDATE:
                            features_test,   descrp_test,  steps_test, df_ = read_data_validate(id_,model=model_)
                        else:
                            
                            features_test,   descrp_test,  steps_test, df_ = read_data(id_,model=model_)
                        features_use[(model_.initial_hour, model_.patch_size)] = [features_test,   descrp_test,  steps_test, df_]
                    
                    matrix=np.stack(features_test[id_]).reshape((-1, model_. CFG['block_size'] //  model_.CFG['patch_size'],  model_. dim))
                    
                    preds =  model_.model.predict(matrix,verbose=0)
                    
                    if model_.TAM_RESHAPE!=1:
                        preds=np.tile(   preds,model_.TAM_RESHAPE).reshape(preds.shape[0],preds.shape[1]*model_.TAM_RESHAPE,2)
               

                    df__=  get_df(preds,df_,model_,TAM_CONSIDER=TAM_CONSIDER)
                    if False:
                        fig=plt.figure()
                        plt.plot(np.arange(len(df__.reshape(-1,2))),df__.reshape(-1,2)[:,0])
                        plt.title(f"ID: {id_}, {model_.model_name}")
                        plt.show()
                    if len(df__)<len(df_total):
                        df_total=df_total[:len(df__)]
                    elif len(df__)>len(df_total):
                         df__=df__[:len(df_total)]
                    df_total+=df__
                    
            if VALIDATE:
                dfs_save.append( (df_total.copy(),df_,id_))

 
            submision = get_events( df_total,df_,id_,patch_size= model_.patch_size,d=19,h=5e-6,TAM_CONSIDER=TAM_CONSIDER)
            submisions.append( submision )
            
            
    import pickle as pkl
    df_save=dfs_save
    file_save ="preds_save.pkl"
    pkl.dump(df_save,open(    file_save ,"wb"))
    del df_save
    gc.collect()

    

    submision=pd.concat(  submisions ) 
    submision["step"]=submision["step"].astype(np.float32)
    

    submision=submision.reset_index(drop=True).reset_index(names='row_id')
    solution=[]
    if VALIDATE:
                events=pd.read_csv("/kaggle/input/child-mind-institute-detect-sleep-states/train_events.csv")
                for id_ in ids_test:
                        events_=events.loc[events.series_id==swapped_dict[id_]].dropna(subset="timestamp")
                        events_["series_id"]=  events_["series_id"].transform(lambda x: dict_ids[x])
                        solution.append(events_)

                solution=pd.concat(solution) .reset_index(drop=True)
                solution.to_csv("solution.csv")
                tolerances= {"onset":[12, 36, 60, 90, 120, 150, 180, 240, 300, 360],"wakeup":[12, 36, 60, 90, 120, 150, 180, 240, 300, 360]}
                print( score(solution.reset_index(drop=True)[["step","series_id","event"]],submision.reset_index(drop=True)[["series_id","event","score","step"]],tolerances,    series_id_column_name="series_id",time_column_name="step",event_column_name="event", score_column_name="score" ))
    return submision

In [32]:
W_BASE=0.03
W_BASE_5=0.03
W_BASE_8=0.005
W_CONV=0.07

In [33]:
model0=ModelSpec(model_name= f"/kaggle/input/modelos-critic/models_5LSTM_lognormal/model_weights_8_fold0_all_/model_weights_8_fold0_all",padding = REPLACE, use_temp = True,  model_type=LINEAR,drop_initial_date=True ,sample_normalize=True,use_tam_consider=True, initial_hour=12,TAM_RESHAPE=10 ,patch_size=10,model_dim=160,weight=W_BASE)
model1=ModelSpec(model_name= f"/kaggle/input/modelos-critic/models_5LSTM_lognormal/model_weights_4_fold1_all_/model_weights_4_fold1_all",padding = REPLACE, use_temp = True,  model_type=LINEAR,drop_initial_date=True ,sample_normalize=True,use_tam_consider=True, initial_hour=12,TAM_RESHAPE=10 ,patch_size=10,model_dim=160,weight=W_BASE)
model2=ModelSpec(model_name= f"/kaggle/input/modelos-critic/models_5LSTM_lognormal/model_weights_6_fold2_all_/model_weights_6_fold2_all",padding = REPLACE, use_temp = True,  model_type=LINEAR,drop_initial_date=True ,sample_normalize=True,use_tam_consider=True, initial_hour=12,TAM_RESHAPE=10 ,patch_size=10,model_dim=160,weight=W_BASE)
model3=ModelSpec(model_name= f"/kaggle/input/modelos-critic/models_5LSTM_lognormal/model_weights_6_fold3_all_/model_weights_6_fold3_all",padding = REPLACE, use_temp = True,  model_type=LINEAR,drop_initial_date=True ,sample_normalize=True,use_tam_consider=True, initial_hour=12,TAM_RESHAPE=10 ,patch_size=10,model_dim=160,weight=W_BASE)
model4=ModelSpec(model_name= f"/kaggle/input/modelos-critic/models_5LSTM_lognormal/model_weights_5_fold4_all_/model_weights_5_fold4_all",padding = REPLACE, use_temp = True,  model_type=LINEAR,drop_initial_date=True ,sample_normalize=True,use_tam_consider=True, initial_hour=12,TAM_RESHAPE=10 ,patch_size=10,model_dim=160,weight=W_BASE)

In [34]:
model0_test=ModelSpec(model_name= f"/kaggle/input/modelos-critic/models_5LSTM_lognormal/model_weights_7_fold0_test_/model_weights_7_fold0_test",padding = REPLACE, use_temp = True,  model_type=LINEAR,drop_initial_date=True ,sample_normalize=True,use_tam_consider=True, initial_hour=12,TAM_RESHAPE=10  ,patch_size=10,model_dim=160,weight=W_BASE)
model1_test=ModelSpec(model_name= f"/kaggle/input/modelos-critic/models_5LSTM_lognormal/model_weights_4_fold1_test_/model_weights_4_fold1_test",padding = REPLACE, use_temp = True,  model_type=LINEAR,drop_initial_date=True ,sample_normalize=True,use_tam_consider=True, initial_hour=12,TAM_RESHAPE=10  ,patch_size=10,model_dim=160,weight=W_BASE)
model2_test=ModelSpec(model_name= f"/kaggle/input/modelos-critic/models_5LSTM_lognormal/model_weights_9_fold2_test_/model_weights_9_fold2_test",padding = REPLACE, use_temp = True,  model_type=LINEAR,drop_initial_date=True ,sample_normalize=True,use_tam_consider=True, initial_hour=12,TAM_RESHAPE=10  ,patch_size=10,model_dim=160,weight=W_BASE)
model3_test=ModelSpec(model_name= f"/kaggle/input/modelos-critic/models_5LSTM_lognormal/model_weights_5_fold3_test_/model_weights_5_fold3_test",padding = REPLACE, use_temp = True,  model_type=LINEAR,drop_initial_date=True ,sample_normalize=True,use_tam_consider=True, initial_hour=12,TAM_RESHAPE=10  ,patch_size=10,model_dim=160,weight=W_BASE)
model4_test=ModelSpec(model_name= f"/kaggle/input/modelos-critic/models_5LSTM_lognormal/model_weights_3_fold4_test_/model_weights_3_fold4_test",padding = REPLACE, use_temp = True,  model_type=LINEAR,drop_initial_date=True ,sample_normalize=True,use_tam_consider=True, initial_hour=12,TAM_RESHAPE=10  ,patch_size=10,model_dim=160,weight=W_BASE)

In [35]:
model0_conv1=ModelSpec(model_name= f"/kaggle/input/models-conv/models_conv6/model_weights_6_fold0_all/model_weights_6_fold0_all",padding = REPLACE, use_temp = True,  model_type=CONV1,drop_initial_date=True ,sample_normalize=True ,initial_hour=12 ,patch_size=1,model_dim=None,use_tam_consider=True,TAM_RESHAPE=8,weight=W_CONV)
model1_conv1=ModelSpec(model_name= f"/kaggle/input/models-conv/models_conv6/model_weights_9_fold1_all/model_weights_9_fold1_all",padding = REPLACE, use_temp = True,  model_type=CONV1,drop_initial_date=True ,sample_normalize=True ,initial_hour=12 ,patch_size=1,model_dim=None,use_tam_consider=True,TAM_RESHAPE=8,weight=W_CONV)
model2_conv1=ModelSpec(model_name= f"/kaggle/input/models-conv/models_conv6/model_weights_9_fold2_all/model_weights_9_fold2_all",padding = REPLACE, use_temp = True,  model_type=CONV1,drop_initial_date=True ,sample_normalize=True ,initial_hour=12 ,patch_size=1,model_dim=None,use_tam_consider=True,TAM_RESHAPE=8,weight=W_CONV)
model3_conv1=ModelSpec(model_name= f"/kaggle/input/models-conv/models_conv6/model_weights_7_fold3_all/model_weights_7_fold3_all",padding = REPLACE, use_temp = True,  model_type=CONV1,drop_initial_date=True ,sample_normalize=True ,initial_hour=12 ,patch_size=1,model_dim=None,use_tam_consider=True,TAM_RESHAPE=8,weight=W_CONV)
model4_conv1=ModelSpec(model_name= f"/kaggle/input/models-conv/models_conv6/model_weights_6_fold4_all/model_weights_6_fold4_all",padding = REPLACE, use_temp = True,  model_type=CONV1,drop_initial_date=True ,sample_normalize=True ,initial_hour=12 ,patch_size=1,model_dim=None,use_tam_consider=True,TAM_RESHAPE=8,weight=W_CONV)
model0_test_conv1=ModelSpec(model_name= f"/kaggle/input/models-conv/models_conv6/model_weights_5_fold0_test/model_weights_5_fold0_test",padding = REPLACE, use_temp = True,  model_type=CONV1,drop_initial_date=True ,sample_normalize=True ,initial_hour=12 ,patch_size=1,model_dim=None,use_tam_consider=True,TAM_RESHAPE=8,weight=W_CONV)
model1_test_conv1=ModelSpec(model_name= f"/kaggle/input/models-conv/models_conv6/model_weights_8_fold1_test/model_weights_8_fold1_test",padding = REPLACE, use_temp = True,  model_type=CONV1,drop_initial_date=True ,sample_normalize=True ,initial_hour=12 ,patch_size=1,model_dim=None,use_tam_consider=True,TAM_RESHAPE=8,weight=W_CONV)
model2_test_conv1=ModelSpec(model_name= f"/kaggle/input/models-conv/models_conv6/model_weights_7_fold2_test/model_weights_7_fold2_test",padding = REPLACE, use_temp = True,  model_type=CONV1,drop_initial_date=True ,sample_normalize=True ,initial_hour=12 ,patch_size=1,model_dim=None,use_tam_consider=True,TAM_RESHAPE=8,weight=W_CONV)
model3_test_conv1=ModelSpec(model_name= f"/kaggle/input/models-conv/models_conv6/model_weights_5_fold3_test/model_weights_5_fold3_test",padding = REPLACE, use_temp = True,  model_type=CONV1,drop_initial_date=True ,sample_normalize=True ,initial_hour=12 ,patch_size=1,model_dim=None,use_tam_consider=True,TAM_RESHAPE=8,weight=W_CONV)
model4_test_conv1=ModelSpec(model_name= f"/kaggle/input/models-conv/models_conv6/model_weights_4_fold4_test/model_weights_4_fold4_test",padding = REPLACE, use_temp = True,  model_type=CONV1,drop_initial_date=True ,sample_normalize=True ,initial_hour=12 ,patch_size=1,model_dim=None,use_tam_consider=True,TAM_RESHAPE=8,weight=W_CONV)


In [36]:
import glob


In [37]:
import glob
models_conv_lstmdropout= [e.replace('.data-00000-of-00001','') for e in glob.glob("/kaggle/input/models-conv/models_conv6_dropout0_lstmdropout/*/*data*")]
models_conv_lstmdropout=[ModelSpec(model_name= e,padding = REPLACE, use_temp = True,  model_type=CONV1_LSTMDROPOUT,drop_initial_date=True ,sample_normalize=True ,initial_hour=12 ,patch_size=1,model_dim=None,use_tam_consider=True,TAM_RESHAPE=8,weight=W_CONV) for e in models_conv_lstmdropout]
models_conv_lstmdropout=list(models_conv_lstmdropout)

In [38]:
import glob
models_lstm_dropout= [e.replace('.data-00000-of-00001','') for e in glob.glob("/kaggle/input/modelos-critic/models_5lstm_lognormal_lstmDropout/*/*data*")]
models_lstm_dropout=[ModelSpec(model_name= e,padding = REPLACE, use_temp = True,  model_type=LINEAR_lstm_dropout,drop_initial_date=True ,sample_normalize=True,use_tam_consider=True, initial_hour=12,TAM_RESHAPE=10 ,patch_size=10,model_dim=160,weight=W_BASE) for e in models_lstm_dropout]
models_lstm_dropout=list(models_lstm_dropout)

In [39]:

if VALIDATE:
    ids_test= [109,144,106,217,234]
    
models_stride_5= [e.replace('.data-00000-of-00001','') for e in glob.glob("/kaggle/input/d/mpp1998/models-critic2/models_5lstm_lognormal_patzh5/*/*data*") ]
models_stride_5=[ModelSpec(model_name= e,padding = REPLACE, use_temp = True,  model_type=LINEAR,drop_initial_date=True ,sample_normalize=True,use_tam_consider=True, initial_hour=12,TAM_RESHAPE=5 ,patch_size=5,model_dim=160,weight=W_BASE, STRIDE=15120) for e in models_stride_5]
models_stride_5=list(models_stride_5)

models_stride_5_gru= [e.replace('.data-00000-of-00001','') for e in glob.glob("/kaggle/input/d/mpp1998/models-critic2/models_5gru_lognormal/*/*data*") ]
models_stride_5_gru=[ModelSpec(model_name= e,padding = REPLACE, use_temp = True,  model_type=GRU,drop_initial_date=True ,sample_normalize=True,use_tam_consider=True, initial_hour=12,TAM_RESHAPE=5 ,patch_size=5,model_dim=160,weight=W_BASE, STRIDE=15120) for e in models_stride_5_gru]
models_stride_5_gru=list(models_stride_5_gru)

models_stride= [e.replace('.data-00000-of-00001','') for e in glob.glob("/kaggle/input/modelos-critic/models_5lstm_lognormal_stride/*/*data*") ]
models_stride=[ModelSpec(model_name= e,padding = REPLACE, use_temp = True,  model_type=LINEAR,drop_initial_date=True ,sample_normalize=True,use_tam_consider=True, initial_hour=12,TAM_RESHAPE=10 ,patch_size=10,model_dim=160,weight=W_BASE, STRIDE=15120) for e in models_stride]
models_stride=list(models_stride)

models_conv= [e.replace('.data-00000-of-00001','') for e in glob.glob("/kaggle/input/models-conv/models_conv6_dropout0/*/*data*") if "fold0" in e or "fold1" in e ]
models_conv=[ModelSpec(model_name= e,padding = REPLACE, use_temp = True,  model_type=CONV1,drop_initial_date=True ,sample_normalize=True ,initial_hour=12 ,patch_size=1,model_dim=None,use_tam_consider=True,TAM_RESHAPE=8,weight=W_CONV) for e in models_conv]
models_conv=list(models_conv)


models_stride_3= [e.replace('.data-00000-of-00001','') for e in glob.glob("/kaggle/input/d/mpp1998/models-critic2/models-5_lstm_lognormlal_path3/*/*data*") ]
models_stride_3=[ModelSpec(model_name= e,padding = REPLACE, use_temp = True,  model_type=LINEAR,drop_initial_date=True ,sample_normalize=True,use_tam_consider=True, initial_hour=12,TAM_RESHAPE=3 ,patch_size=3,model_dim=160,weight=W_BASE, STRIDE=15120) for e in models_stride_3]
models_stride_3=list(models_stride_3)


models_stride_4= [e.replace('.data-00000-of-00001','') for e in glob.glob("/kaggle/input/d/mpp1998/models-critic2/models_5lstm_lognroaml_path4/*/*data*") ]
models_stride_4=[ModelSpec(model_name= e,padding = REPLACE, use_temp = True,  model_type=LINEAR,drop_initial_date=True ,sample_normalize=True,use_tam_consider=True, initial_hour=12,TAM_RESHAPE=4 ,patch_size=4,model_dim=160,weight=W_BASE, STRIDE=15120) for e in models_stride_4]
models_stride_4=list(models_stride_4)

models_stride_6= [e.replace('.data-00000-of-00001','') for e in glob.glob("/kaggle/input/d/mpp1998/models-critic2/models_5lstm_lognromal_path6/*/*data*") ]
models_stride_6=[ModelSpec(model_name= e,padding = REPLACE, use_temp = True,  model_type=LINEAR,drop_initial_date=True ,sample_normalize=True,use_tam_consider=True, initial_hour=12,TAM_RESHAPE=6 ,patch_size=6,model_dim=160,weight=W_BASE, STRIDE=15120) for e in models_stride_6]
models_stride_6=list(models_stride_6)


models_6= [e.replace('.data-00000-of-00001','') for e in glob.glob("/kaggle/input/models-conv/models_conv_droput0_path6/*/*data*")  if "fold0" in e or "fold1" in e]
models_6=[ModelSpec(model_name= e,padding = REPLACE, use_temp = True,  model_type=CONV_6,drop_initial_date=True ,sample_normalize=True ,initial_hour=12 ,patch_size=1,model_dim=None,use_tam_consider=True,TAM_RESHAPE=6,weight=W_CONV) for e in models_6]
models_6=list( models_6)


models_4= [e.replace('.data-00000-of-00001','') for e in glob.glob("/kaggle/input/models-conv2/models_conv_dropout0_patxh5/*/*data*") if "fold0" in e or "fold1" in e]
models_4=[ModelSpec(model_name= e,padding = REPLACE, use_temp = True,  model_type=CONV_5,drop_initial_date=True ,sample_normalize=True ,initial_hour=12 ,patch_size=1,model_dim=None,use_tam_consider=True,TAM_RESHAPE=4,weight=W_CONV) for e in models_4]
models_4=list(models_4)



if len(ids_test)!=3:
        
    for model in [ models_stride_5+models_stride + models_4+ models_6+ models_conv ]:
        submision=predict_model(model,TAM_CONSIDER=5)
else:
    for model in [models_conv]:

            submision=predict_model(model,TAM_CONSIDER=5)

100%|██████████| 3/3 [00:39<00:00, 13.12s/it]


In [40]:
dfs_save=pkl.load(open("/kaggle/working/preds_save.pkl","rb"))

In [41]:
if VALIDATE:
                solution=[] 
                events=pd.read_csv("/kaggle/input/child-mind-institute-detect-sleep-states/train_events.csv")
                for id_ in ids_test:
                        events_=events.loc[events.series_id==swapped_dict[id_]].dropna(subset="timestamp")
                        events_["series_id"]=  events_["series_id"].transform(lambda x: dict_ids[x])
                        solution.append(events_)

                solution=pd.concat(solution) .reset_index(drop=True)
                solution.to_csv("solution.csv")
                tolerances= {"onset":[12, 36, 60, 90, 120, 150, 180, 240, 300, 360],"wakeup":[12, 36, 60, 90, 120, 150, 180, 240, 300, 360]}

In [42]:
submision

,row_id,step,event,series_id,score
0,0,5.0,wakeup,0402a003dae9,0.000012
1,1,149.0,wakeup,0402a003dae9,0.000009


In [43]:
if False and VALIDATE:
    #for h in np.arange(5e-7,5e-5,5e-7):
    for d  in [28,32,40]:
        submisions=[]
        h=5e-7
        #d=8
        for (df_total,df_,id_) in dfs_save:
            submision = get_events( df_total,df_,id_,patch_size= 10,h=h,d=d)
            submisions.append( submision )

        submision=pd.concat(  submisions ) 
        submision["step"]=submision["step"].astype(np.float32)
        print(h,d, score(solution.reset_index(drop=True)[["step","series_id","event"]],submision.reset_index(drop=True)[["series_id","event","score","step"]],tolerances,    series_id_column_name="series_id",time_column_name="step",event_column_name="event", score_column_name="score" ))
    

In [44]:
submision.to_csv('submission.csv', index=False)
submision

,row_id,step,event,series_id,score
0,0,5.0,wakeup,0402a003dae9,0.000012
1,1,149.0,wakeup,0402a003dae9,0.000009
